<a href="https://colab.research.google.com/github/siddhant-parmar/CMPE_255_Group_4/blob/main/Food_poisoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np 

review_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_review.csv")

In [ ]:
busi_data = pd.read_csv("/content/drive/My Drive/cmpe 255/yelp_business.csv")

In [ ]:
review_data = review_data[['business_id', 'text', 'stars']]

print(review_data.shape)
review_data.head()

(5261668, 3)


,business_id,text,stars
0,AEx2SYEUJmTxVVB18LlCwA,Super simple place but amazing nonetheless. It...,5
1,VR6GpWIda3SfvPC-lg9H3w,Small unassuming place that changes their menu...,5
2,CKC0-MOWMqoeWf6s-szl8g,Lester's is located in a beautiful neighborhoo...,5
3,ACFtxLv8pGrrxMm6EgjreA,Love coming here. Yes the place always needs t...,4
4,s2I_Ni76bjJNK9yG60iD-Q,Had their chocolate almond croissant and it wa...,4


In [ ]:
print(busi_data.shape)
busi_data = busi_data[busi_data.categories.str.contains('restaurant|food', na=False)]

print(busi_data.shape)
busi_data = busi_data[['business_id', 'name']]
busi_data.head()

(174567, 13)
(2313, 13)


,business_id,name
44,BnuzcebyB1AfxH0kjNWqSg,"""Carrabba's Italian Grill"""
101,VdlPZg2NAu8t8GkdbPLecg,"""Tandoori Times Indian Bistro"""
107,QkG3KUXwqZBW18A9k1xqCA,"""Red Lobster"""
163,xMP2oDrKFqyUrN0Uww85EQ,"""Dockside Seafood Market"""
274,dPxZI9lrKTl5dvFfnb1_Ig,"""Trattoria Italia"""


In [ ]:
# data = data[data.business_id.str.contains()]

In [ ]:
review_data['business_id'] = review_data['business_id'].astype(str)
busi_data['business_id'] = busi_data['business_id'].astype(str)

data = review_data.merge(busi_data, on='business_id')
print(data.shape)
data.head()

(248709, 4)


,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,This place reminds of St Lawrence Market in To...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,A very quaint farmers market right by the wate...,4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,A must on any Montreal visit if nothing else b...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,"Marche Atwater is a cute little market, they h...",4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,Le Coin Gourmand carries the best store bought...,4,"""Marché Atwater"""


In [ ]:
# counting null values in the dataset

data.isnull().sum()

business_id    0
text           0
stars          0
name           0
dtype: int64

In [ ]:
import nltk 
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

data['text'] = data['text'].astype(str)
#print(data.head())
data['text'] = data['text'].str.lower()
#print(data.head())

# stop word removal
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,"quaint farmers market right waterfront, makes ...",4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,"marche atwater cute little market, booths outs...",4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater"""


In [ ]:
import string
import nltk
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WhitespaceTokenizer
import re

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None
    
    
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    # lemmatize text
    pos_tagged = nltk.pos_tag(nltk.word_tokenize(text))

    wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
    # print(wordnet_tagged)

    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            # if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:       
            # else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    text = " ".join(lemmatized_sentence)
    return(text)


def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def clean_text(text):
    # remove html tags
    text = remove_html_tags(text)
    # lower text
    text = text.lower()
    # unicode characters
    text = text.encode('ascii', 'ignore').decode()
    # remove digits
    text = ''.join([i for i in text if not i.isdigit()])
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove stop words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split(' ') if word not in stop_words])
    return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data['text'] = data.text.apply(clean_text)
data.head()

,business_id,text,stars,name
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater"""
1,RWGI8u00x5GghYCEzpYrIg,quaint farmers market right waterfront makes g...,4,"""Marché Atwater"""
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater"""
3,RWGI8u00x5GghYCEzpYrIg,marche atwater cute little market booths outsi...,4,"""Marché Atwater"""
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater"""


In [ ]:
words = ["discomfort","ache","death","allergy","diarrhoea", "stomach", "loosemotion", "sick", "vomit", "nausea", "throw up", "threw up", "unhygienic", "pain", "flu", "stale", "poision"]

for w in words:
  w = lemmatize_text(w);

In [ ]:
print(words)

['discomfort', 'ache', 'death', 'allergy', 'diarrhoea', 'stomach', 'loosemotion', 'sick', 'vomit', 'nausea', 'throw up', 'threw up', 'unhygienic', 'pain', 'flu', 'stale', 'poision']


In [ ]:
# creating a new feature for food poisoning based on words seen in review text
for index,row in data.iterrows() :
    if any(word in row['text'] for word in words) and row['stars'] < 3:
        data.loc[index,'FP'] = 1
    else:
        data.loc[index,'FP'] = 0

X = data[['text','stars']]   #convert text to some word2vec or onehotencoding
Y = data['FP']

data.head()

,business_id,text,stars,name,FP
0,RWGI8u00x5GghYCEzpYrIg,place reminds st lawrence market toronto veggi...,4,"""Marché Atwater""",0.0
1,RWGI8u00x5GghYCEzpYrIg,quaint farmers market right waterfront makes g...,4,"""Marché Atwater""",0.0
2,RWGI8u00x5GghYCEzpYrIg,must montreal visit nothing else gawk longingl...,5,"""Marché Atwater""",0.0
3,RWGI8u00x5GghYCEzpYrIg,marche atwater cute little market booths outsi...,4,"""Marché Atwater""",0.0
4,RWGI8u00x5GghYCEzpYrIg,le coin gourmand carries best store bought chi...,4,"""Marché Atwater""",0.0


In [ ]:
fp_data = data[data['FP'] == 1.0] 
print(fp_data.shape)
fp_data.head()

(4124, 5)


,business_id,text,stars,name,FP
191,IBZ6e3P0OqcPXVjvGlpueQ,cautionary tale potential diners well restaura...,1,"""Off The Hook Gourmet""",1.0
338,ym5FOqL6tGbgynzwKMHZuw,original bad review removed lack specificity o...,1,"""Crab Corner""",1.0
348,ym5FOqL6tGbgynzwKMHZuw,roaches store roach came food told manager imm...,1,"""Crab Corner""",1.0
359,ym5FOqL6tGbgynzwKMHZuw,love eating snow crab king crab dungeness crab...,2,"""Crab Corner""",1.0
387,ym5FOqL6tGbgynzwKMHZuw,days since visit going let bad experience go ...,1,"""Crab Corner""",1.0


In [ ]:
from sklearn.model_selection import train_test_split

# split the data into train and test set
train, test = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
print(train.shape)
print(test.shape)

train.head()
test.head()

(198967, 5)
(49742, 5)


,business_id,text,stars,name,FP
133803,R_ZlcX46pPdjhjmfd043LA,made reservations pm knowing show get around p...,1,"""Top of the World""",0.0
14053,ibkwcLZwaJ1pnPJVaI_-uA,swadee bomb im thai food lover ive hopped arou...,4,"""Swaddee Thai Cuisine""",0.0
66869,rvnTFEq7R2ZHg0_CYbBNFw,cant believe ive going vegas years went place...,5,"""Village Seafood Buffet""",0.0
101377,vOMDU31gdylrzBhAKC9QbA,came friend super super picky sushi shout mel...,5,"""Sushi Hiroyoshi Japanese Cuisine""",0.0
86041,RZFryvOkznd4jtpj8PbSzg,living southern california ive pretty good sus...,5,"""Sushi Style""",0.0


In [ ]:
# just for our reference
from fastai.imports import *
data_dir = "/content/drive/My Drive/cmpe 255/train-test-data"
# determine the path where to save the train and test file
test_r_path = Path(data_dir, 'test_review.csv')
test_fp_path = Path(data_dir, 'test_expected.csv')

# save the train and test file
# again using the '\t' separator to create tab-separated-values files
test.to_csv(test_r_path, columns=['text'],  index=False)
test.to_csv(test_fp_path, columns=['FP'],  index=False)

In [ ]:
# vectorize before passing to knn classifier
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1, 1), min_df=0.001)
train_features = vectorizer.fit_transform(train["text"])
test_features = vectorizer.transform(test["text"])

In [ ]:
train_labels = train["FP"]

In [ ]:
# knn classifier
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=150, algorithm='auto')
classifier.fit(train_features, train_labels)

KNeighborsClassifier(n_neighbors=150)

In [ ]:
predicted_labels = classifier.predict(test_features)
print("knn classification done.")

knn classification done.


In [ ]:
with open(data_dir + '/output_labels.csv', 'w') as f:
    for item in predicted_labels:
        if item > 0:
            f.write('+1\n')
        elif item == 0:
            review_option = ["+1", "-1"]
            random_choice = random.choice(review_option)
            f.write(random_choice + '\n')
        else:
            f.write('-1\n')
        f.close
print("end")

end


In [ ]:
def find_accuracy(predicted_labels, actual_labels):
    accuracy = 0
    wrong_fp = 0
    wrong_nfp = 0

    i = 0
    l = len(actual_labels)
    while i < l:
        if actual_labels[i] == predicted_labels[i]:
            accuracy += 1
        elif actual_labels[i] == 1.0:
            wrong_fp += 1
        else:
            wrong_nfp += 1
        i += 1

    print("Accurately Identified:", accuracy, "Percentage:", accuracy * 100 / l)
    print("Wrongly Identified as Food Poisoning review:", wrong_fp, "Percentage:", wrong_fp * 100 / l)
    print("Wrongly Identified as Non-food poisoning review:", wrong_nfp, "Percentage:", wrong_nfp * 100 / l)

In [ ]:
test_labels = test["FP"].to_numpy()

print(test_labels)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
print(predicted_labels)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
# finding Accuracy
find_accuracy(predicted_labels, test_labels)

Accurately Identified: 48970 Percentage: 98.44799163684613
Wrongly Identified as Food Poisoning review: 771 Percentage: 1.5499979896264726
Wrongly Identified as Non-food poisoning review: 1 Percentage: 0.002010373527401391


In [ ]:
print(test.head())

                   business_id  ...   FP
133803  R_ZlcX46pPdjhjmfd043LA  ...  0.0
14053   ibkwcLZwaJ1pnPJVaI_-uA  ...  0.0
66869   rvnTFEq7R2ZHg0_CYbBNFw  ...  0.0
101377  vOMDU31gdylrzBhAKC9QbA  ...  0.0
86041   RZFryvOkznd4jtpj8PbSzg  ...  0.0

[5 rows x 5 columns]


In [ ]:
predicted_labels = pd.DataFrame(predicted_labels)
test = pd.concat([test,predicted_labels])

count = test.groupby(['business_id', 'FP'], as_index=False).size()
count = pd.DataFrame(count)
print(count.head())
print(count.isnull().sum())
print()

              business_id   FP  size
0  -06-Fdi30wJx-JA8P7CAng  0.0     5
1  -1m9o3vGRA8IBPNvNqKLmA  0.0     3
2  -1uVyyy3UXNA9WoPucNniA  0.0     6
3  -3zffZUHoY8bQjGfPSoBKQ  0.0   122
4  -3zffZUHoY8bQjGfPSoBKQ  1.0     2
business_id    0
FP             0
size           0
dtype: int64


In [ ]:
percentages = count.groupby('business_id').agg({'size': 'sum'})
print(percentages)

                        size
business_id                 
-06-Fdi30wJx-JA8P7CAng     5
-1m9o3vGRA8IBPNvNqKLmA     3
-1uVyyy3UXNA9WoPucNniA     6
-3zffZUHoY8bQjGfPSoBKQ   124
-6h3K1hj0d4DRcZNUtHDuw    94
...                      ...
zikXPy7iaEIs9kMmhtB0EA     2
znBAFVFRMvdUvJ7eJY_vjA     3
ztP466jMUMtqLwwHqXbk9w    75
zw74kL1IvT65yRvNLx5UxA     4
zyPGYeXF4XKCqNN1pjFWhg    19

[2120 rows x 1 columns]


In [ ]:
pp = percentages.merge(count, on='business_id')
#print(pp)

is_FP1 =  pp['FP']==1.0
final_perc = pp[is_FP1]
final_perc

                 business_id  size_x   FP  size_y
0     -06-Fdi30wJx-JA8P7CAng       5  0.0       5
1     -1m9o3vGRA8IBPNvNqKLmA       3  0.0       3
2     -1uVyyy3UXNA9WoPucNniA       6  0.0       6
3     -3zffZUHoY8bQjGfPSoBKQ     124  0.0     122
4     -3zffZUHoY8bQjGfPSoBKQ     124  1.0       2
...                      ...     ...  ...     ...
2596  zikXPy7iaEIs9kMmhtB0EA       2  0.0       2
2597  znBAFVFRMvdUvJ7eJY_vjA       3  0.0       3
2598  ztP466jMUMtqLwwHqXbk9w      75  0.0      75
2599  zw74kL1IvT65yRvNLx5UxA       4  0.0       4
2600  zyPGYeXF4XKCqNN1pjFWhg      19  0.0      19

[2601 rows x 4 columns]


,business_id,size_x,FP,size_y
4,-3zffZUHoY8bQjGfPSoBKQ,124,1.0,2
6,-6h3K1hj0d4DRcZNUtHDuw,94,1.0,2
8,-8Oaw94x2pYJCVP8ZtuTTw,18,1.0,1
10,-8ZiMXZReeTD3kwEvS0Lww,22,1.0,1
15,-INdswGdyRyy72xXHalNbg,31,1.0,1
...,...,...,...,...
2560,ym5FOqL6tGbgynzwKMHZuw,75,1.0,1
2563,ymtJtXwsJUClLNDepV1piQ,64,1.0,1
2574,z7Sg-WFc9Y1s-07Cb6-WZQ,1,1.0,1
2577,z9lOfnUorcbWR_xS0DSeHw,40,1.0,1


In [ ]:
Bus_prob = []

for index,row in final_perc.iterrows():
  percent = row['size_y'] / row['size_x']
  #print(df)
  Bus_prob.append({'business_id': row['business_id'], 'Percentage': round(percent*100,2)})

Bus_prob = pd.DataFrame(Bus_prob)
pd.set_option("display.max_rows", None, "display.max_columns", None)

sorted_prob = Bus_prob.sort_values(by = 'Percentage', ascending=False)

In [ ]:
sorted_prob

,business_id,Percentage
270,X0UQortyd6N6zFBVisY_gw,100.00
29,2g4jkEMwQxIwRqIEXiX0tg,100.00
286,Z79Ub6ZR7eBzyAD4E3UOMw,100.00
83,A0dOr_yTQO0PyKPNzfz52Q,100.00
368,hmUkDAjKxB1et3hVjhw9jg,100.00
353,gNg1GXdESMs_ChlZKR7TFQ,100.00
488,z7Sg-WFc9Y1s-07Cb6-WZQ,100.00
279,YCdLw_qAjZyG1lbcvJG8_Q,100.00
320,c0mV9byVqQMERBojaFmuWA,100.00
321,c7aifX_T8ztb35YlL-pcuw,100.00


In [ ]:
Final_FP = sorted_prob[sorted_prob["Percentage"] > 30.00]
print(Final_FP)
print(Final_FP.shape[0]," restaurants have more than 30% chance of giving Food Poisoning to their customers")

,business_id,Percentage
270,X0UQortyd6N6zFBVisY_gw,100.00
29,2g4jkEMwQxIwRqIEXiX0tg,100.00
286,Z79Ub6ZR7eBzyAD4E3UOMw,100.00
83,A0dOr_yTQO0PyKPNzfz52Q,100.00
368,hmUkDAjKxB1et3hVjhw9jg,100.00
353,gNg1GXdESMs_ChlZKR7TFQ,100.00
488,z7Sg-WFc9Y1s-07Cb6-WZQ,100.00
279,YCdLw_qAjZyG1lbcvJG8_Q,100.00
320,c0mV9byVqQMERBojaFmuWA,100.00
321,c7aifX_T8ztb35YlL-pcuw,100.00
